# Basics of Mobile Robotics: Project - Global Nav


# Author : PB

In [1]:
import numpy as np
import control as ct
import control.optimal as opt
import matplotlib.pyplot as plt
import scipy.optimize as scopt
from typing import Sequence
from src import model,thymio

## GLOBAL NAVIGATION IMPLEMENTATION

I decided to develop and test my global navigation on this notebook before implementing it in the folder

In [ ]:
# USE OF MPC

#	X    -----------------      u       ---------
#	---> | MPC Regulator |------------> | Plant |----------> Output
#	|    -----------------              ---------   |
#	|           ^                                   |
#	|           | (xs, us)                          |
#	|           |                                   |
#	|           |       ------------                |
#	|           --------| target   |<----------------
#	|                   | selector |<-- reference   |
#	|                   ------------                |
#	|                                               |
#	-------------------------------------------------

Initial parameters

In [ ]:
# initialization block
Tf = 10
min_velocity = 0
max_velocity = 1
min_deviation_phi = -1
max_deviation_phi = 1

tps = np.linspace(0, Tf, Tf)


#create the data point
xaxis = np.linspace(0.0, 10.0, num=100)
yaxis = 20*np.sin(xaxis/5)

Length = len(xaxis)
Sequence = []
for i in range(Length):
    p = model.Point(xaxis[i], yaxis[i])
    Sequence.append(p)
#Sequence = np.array(Sequence)

#plt.plot(tps, 3*np.sin(tps/3))
plt.plot(xaxis, yaxis)

x0 = [0., 0., 0.2]; u0 = [np.sqrt((tps[1]-tps[0])**2 + (np.sin(tps[1])-np.sin(tps[0]))**2), (tps[1]-tps[0])/Tf]
xf = [Sequence[-1].x, Sequence[-1].y, 0.]; uf = [10., 0.]

def lafonc(x):
    return 3*np.sin(x/3)

Q = np.array([[0.1, 10, 0],[0, 10, 0], [0, 0, 0.1]])          # don't turn too sharply
R = np.diag([1, 1])               # keep inputs small
P = np.diag([1000, 1000, 1000])   # get close to final point


plt.plot(x0[0], x0[1], marker="o")
plt.plot(xf[0], xf[1], marker="o")

Useful functions

In [ ]:
def vehicle_update(t, x, u, params):
    # Get the parameters for the model
    l = params.get('wheelbase', 3.)         # vehicle wheelbase
    #phimax = params.get('maxsteer', 0.5)    # max steering angle (rad)
    
    # Saturate the steering input
    phi = np.clip(u[1], min_deviation_phi, max_deviation_phi)

    # Return the derivative of the state
    return np.array([
        np.cos(x[2]) * u[0],            # xdot = cos(theta) v
        np.sin(x[2]) * u[0],            # ydot = sin(theta) v
        (u[0] / l) * np.tan(phi)        # thdot = v/l tan(phi)
    ])



def vehicle_output(t, x, u, params):
    return x                            # return x, y, theta (full state)

# Define the vehicle steering dynamics as an input/output system
vehicle = ct.NonlinearIOSystem(
    vehicle_update, vehicle_output, states=3, name='vehicle',
    inputs=('v', 'phi'), outputs=('x', 'y', 'theta'))

First iteration test

In [ ]:
LengthHorizon = 3

traj_cost = opt.quadratic_cost(vehicle, Q, R, x0=xf, u0=uf)
term_cost = opt.quadratic_cost(vehicle, P, 0, x0=xf)

def fun(x,u):
    index = min(range(len(Sequence)), key=lambda i: abs(Sequence[i].x-x[0]))
    return x[1]-Sequence[index].y
lb = -0.1
ub = 0.1

#constraints = [{"type": "ineq", "fun": lambda x: x[0] - 3*np.sin(x[1]/3)+2},{"type": "ineq", "fun": lambda x: -x[0] + 3*np.sin(x[1]/3)+2}]
#constraints = [(scopt.NonlinearConstraint, fun, lb, ub), opt.input_range_constraint(vehicle, [min_velocity, min_deviation_phi], [max_velocity, max_deviation_phi]) ]
WidthTunnel = 0.5
# Sequence[min(range(len(Sequence)), key=lambda i: abs(Sequence[i].x-x[0]))].y-WidthTunnel
constraints = [opt.state_range_constraint(vehicle, [0, 3, -np.inf], [10, 10, np.inf]), opt.input_range_constraint(vehicle, [min_velocity, min_deviation_phi], [max_velocity, max_deviation_phi])]
print(constraints)  

horizon = np.linspace(0, LengthHorizon, LengthHorizon, endpoint=True)
result = opt.solve_ocp(vehicle, horizon, x0, traj_cost, constraints, terminal_cost=term_cost, initial_guess=u0, log=False)
#result = opt.create_mpc_iosystem(vehicle, horizon, x0, traj_cost, constraints, terminal_cost=term_cost, initial_guess=u0, log=False)

# Simulate the system dynamics (open loop)
resp = ct.input_output_response(
    vehicle, horizon, result.inputs, x0,
    t_eval=np.linspace(0, LengthHorizon, LengthHorizon))
t, y, u = resp.time, resp.outputs, resp.inputs

# Plot
plt.plot(x0[0], x0[1], 'ro', xf[0], xf[1], 'ro')
plt.plot(xaxis, yaxis, 'g')
plt.plot(y[0], y[1], 'o-')
plt.show()
print(y)

Optimization loop

In [ ]:
# still working on this part

Xfinal = np.zeros((Tf,3))

LengthHorizon = 3
eps = 0.5
y = np.zeros((3,LengthHorizon))

def fun(x,u):
    index = min(range(len(Sequence)), key=lambda i: abs(Sequence[i].x-x[0]))
    return x[1]-Sequence[index].y
lb = -0.1
ub = 0.1

ite = 0

while (abs(y[0,LengthHorizon-1]-xf[0])>eps or abs(y[1,LengthHorizon-1]-xf[1])>eps) and ite<15:
    print("step %s" % ite)
    if (ite+LengthHorizon<Tf):
        index = ite+LengthHorizon
    else:
        index = Tf-1

    #xgoal = [tps[index], 3*np.sin(tps[index]/3), (tps[index]-tps[index-1])/Tf]
    traj_cost = opt.quadratic_cost(vehicle, Q, R, x0=xf, u0=uf)
    term_cost = opt.quadratic_cost(vehicle, P, 0, x0=xf)
     

    constraints = [(scopt.NonlinearConstraint, fun, lb, ub), opt.input_range_constraint(vehicle, [min_velocity, min_deviation_phi], [max_velocity, max_deviation_phi]) ]
    #constraints = []

    horizon = np.linspace(0, LengthHorizon, LengthHorizon, endpoint=True)
    result = opt.solve_ocp(vehicle, horizon, x0, traj_cost, constraints,terminal_cost=term_cost, initial_guess=u0)

    # Simulate the system dynamics (open loop)
    resp = ct.input_output_response(
        vehicle, horizon, result.inputs, x0,
        t_eval=np.linspace(0, LengthHorizon, LengthHorizon))
    t, y, u = resp.time, resp.outputs, resp.inputs
    
    x0 = y[:,1]

    Xfinal[ite,:] = y[:,1]
    plt.plot(y[0], y[1], 'o-')
    plt.legend("%s" %ite)
    ite += 1

for j in range(LengthHorizon):
    Xfinal[Tf-LengthHorizon+j,:] = y[:,len(y)-LengthHorizon+j]

plt.plot(xaxis, yaxis, 'g')
#plt.plot(Xfinal[0:ite+LengthHorizon,0],Xfinal[0:ite+LengthHorizon,1],'yo--',linewidth=5.0)
plt.show()

PID

In [9]:
#at last, where the added value of a MPC controller ? hmmm, difficult to see
# so let's go PID

#to put in the initialization file
PID_errors = np.zeros((1,3))

def follow_path(robot: model.Robot, path: Sequence[model.Point]) -> model.MotorSpeed:
    ### Access the robot's pose
    # robot.position.x
    # robot.position.y
    # robot.position.z

    #find the closest index in the path array with the current robot position
    index = min(range(len(path)), key=lambda i: abs(path[i].x-robot.position.x))

    # Calcul of the perpendicular distance between the robot position and the path
    PathPosH, PathPosL = index+1,index
    if index+1>=len(path)-1:
        PathPosH, PathPosL = len(path)-1,len(path)-2  
    p1 = np.array([path[index].x, path[index].y]) #previous correct position on the path
    p2 = np.array([path[index+1].x, path[index+1].y]) #previous correct position on the path
    p3 = np.array([robot.position.x, robot.position.y]) #array of the robot position

    d=abs(np.cross(p2-p1,p3-p1)/np.linalg.norm(p2-p1))

    # error > 0 robot over the correct path
    # error > à robot below the correct path
    if p3(1)>=p1(1) or p3(1)>=p2(1):
        error = d # error of position
    else:
        error = -d

    # PID correction
    PID_errors[0], PID_errors[1]; PID_errors[2] = error, PID_errors[1]+error, error-PID_errors[0]
    # basically it does what is below
    #proportional, p = error
    #integrator, i += p
    #derivative, d = p - lp
    #update previous error, lp = p

    max_speed = 255
    #Ziegler-Nichols method for PID
    Ku, Tu = 1, 1
    PID_coefficients = np.array([0.6*Ku,1.2*Ku/Tu,3*Ku*Tu/40]) #[Kp, Ki, Kd]
    #adding the correction to the base_speed for the left and right motor
    correction = int(PID_coefficients*PID_errors)
    
    # checking the direction of the robot to reduce the speed of the closest motor engine to the correct path
    if robot.angle>0 and robot.angle<np.pi:
        if error>0:
            rspeed = max_speed - correction
            lspeed = max_speed
        else:
            rspeed = max_speed
            lspeed = max_speed - correction
    else:
        if error>0:
            rspeed = max_speed
            lspeed = max_speed - correction
        else:
            rspeed = max_speed - correction 
            lspeed = max_speed       

    #restricting speed of motors between 255 and -255
    if (rspeed > 255):
        rspeed = 255    
    if (lspeed > 255):
        lspeed = 255    
    if (rspeed < -255):
        rspeed = -255    
    if (lspeed < -255):
        lspeed = -255

    ### Return the command to send to the motors
    return model.MotorSpeed(lspeed, rspeed)
    # pass

In [7]:
p1=np.array([0,0])
p2=np.array([10,10])
p3=np.array([5,7])
d=np.cross(p2-p1,p3-p1)/np.linalg.norm(p2-p1)
print(d)


1.414213562373095
